# Analysis of simulated data


TODO add here a brief description of your project or analysis. Explain the purpose, goals, or context of the notebook here.

## Table of Contents
- [1. Chapter 1 - Introduction](#chapter-1-introduction)
- [2. Chapter 2 - R packages](#chapter-2-R-packages)
- [3. Chapter 3 - Input specification](#chapter-3-Input-specification)
- [4. Chapter 4 - Supermatrix and Supertree trees VS true topologies](#chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies)
- [5. Chapter 5 - BigTree VS true topologies](#chapter-5-BigTree-VS-true-topologies)
- [6. Chapter 6 - RF distance summaries](#chapter-6-RF-distance-summaries)
    - [6.1 Subchapter 6.1 - Table 1](#subchapter-6_1-Table-1)
- [7. Chapter 7 - Running times](#chapter-7-Running-times)
    - [7.1 Subchapter 7.1 - Figure 2](#subchapter-7_1-Figure-2)

<a id="chapter-1-introduction"></a>

## Chapter 1 - Introduction

TODO add description of notebook and how to use it.


<a id="chapter-2-R-packages"></a>

## Chapter 2 - R packages

Here is the list of packages needed for this notebook.

In [1]:
library("phangorn")
library("ggplot2")
library("reshape2")
library("geiger")
library("adephylo")
library("phytools")
library("rlist")
library("plotrix")
library("plyr")
library("dplyr")
library("hrbrthemes")
library("rstatix")
library("gplots")
library("viridis")
library('gtools')
library(gridExtra)
library(grid)
library("ggforce")
library("scales")

Loading required package: ape

Loading required package: phytools

Loading required package: maps

Loading required package: ade4


Attaching package: ‘plotrix’


The following object is masked from ‘package:phytools’:

    rescale



Attaching package: ‘plyr’


The following object is masked from ‘package:maps’:

    ozone



Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:ape’:

    where


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘gplots’


The following object is masked from ‘package:plotrix’:

    plotC

TODO make a chapter for aqll custom functions

<a id="chapter-3-Input-specification"></a>

## Chapter 3 - Input specification

Change the parent_of_all_dir variable to give the location of the input. All input files are expected to be under it. Otherwise change the single input location to you liking.<br>
All input files or directory paths are listed here.

In [2]:
# the location of the directory that contains all inputs for analyses 
parent_of_all_dir = "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/"

# The directory (one or more) that contain all the results from the main.nf pipeline run. It can be a glob path.
#dir_containing_all_pipe_results = "../results" # todo make this disappear once analysis are ready
dir_containing_all_pipe_results = paste0(parent_of_all_dir, "rerun_25_species_corrected_branches/rerun_sim_b_factor*")

#  TODO describe what this file does
reference_tree_codefile = file.path(parent_of_all_dir, "ref_tree.code")

# get the species/organism list of names used by the main.nf pipeline
species_name_file = file.path(parent_of_all_dir, "orthologs_org_ids_to_concatenate")


# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

# Create the 'figures' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("figures")) {
    dir.create("figures")
}

# Create the 'tables' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("tables")) {
    dir.create("tables")
}

<a id="chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies"></a>

## Chapter 4 - Supermatrix and Supertree trees VS true topologies

TODO add description 

In [11]:
# setting global lists that eill hold all family level values
RF_avg_list = list()
collect_branch_len_supermatrix_ml = c()
collect_diameter_supermatrix_ml = c()

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # this will contain the mean branch length for all true/reference paralog trees for each gene family
    avg_branch_length = c()

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # get all units trees for all combination of methods 
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # Get all units trees (250 = 25 organism * 10 samples/statistical repeats) for the combination Minimun Evolution (ME) + supermatrix method (SM). 
        # We unroot every tree and fully resolve it to use the RF.dist package and others. 
        all_units_file  = file.path(m, "results/rerun", "all_units.nwk")
        all_units_trees = read.tree(all_units_file, keep.multi=TRUE)
        all_units_trees = unroot(all_units_trees)
        
        # check if there any non-bynary (consensus trees) among the 250 trees present in the file above
        if (length(which(is.binary(all_units_trees) == FALSE)) != 0) {
            
            # if there are non-binary trees, iterate through them and transformed into a binary one using multi2di package
            for (tree in which(is.binary(all_units_trees) == FALSE)) {
                all_units_trees[[tree]] = multi2di(all_units_trees[[tree]])
            }
        }

        # Get all units trees (250) ME + supertree (ST) method and handle non-binary trees as above
        all_superfine_file  = file.path(m, "results/rerun", "all_units_superfine_trees.nwk")
        all_superfine_trees = read.tree(all_superfine_file, keep.multi=TRUE)
        all_superfine_trees = unroot(all_superfine_trees)
        if (length(which(is.binary(all_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_superfine_trees) == FALSE)) {
                all_superfine_trees[[tree]] = multi2di(all_superfine_trees[[tree]])
            }
        }

        # Get all units trees (250) Maximum Likelihood (ML) + SM 
        all_units_raxml_file  = file.path(m, "results/rerun", "all_units_RAxML_trees.nwk")
        all_units_raxml_trees = read.tree(all_units_raxml_file, keep.multi=TRUE)
        all_units_raxml_trees = unroot(all_units_raxml_trees)
        if (length(which(is.binary(all_units_raxml_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_units_raxml_trees) == FALSE)) {
                all_units_raxml_trees[[tree]] = multi2di(all_units_raxml_trees[[tree]])
            }
        }

        # take only the last 10 of the 250 above trees (unit25's 10 samples/replicates)(unit25 = all 25 single units concatenated) and collect branch lengths (each branch in tree, 27 in total 2*15-3) and diameter (using patristic distance)
        for(mytree in 241:length(all_units_raxml_trees)) {
            collect_branch_len_supermatrix_ml = c(collect_branch_len_supermatrix_ml, all_units_raxml_trees[[mytree]]$edge.length)
            collect_diameter_supermatrix_ml = c(collect_diameter_supermatrix_ml, max(distTips(all_units_raxml_trees[[mytree]], method = 'patristic')))
        }

        # Get all units trees (250) ML + ST
        all_raxml_superfine_file  = file.path(m, "results/rerun", "all_units_RAxML_superfine_trees.nwk")
        all_raxml_superfine_trees = read.tree(all_raxml_superfine_file, keep.multi=TRUE)
        all_raxml_superfine_trees = unroot(all_raxml_superfine_trees)
        if (length(which(is.binary(all_raxml_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_raxml_superfine_trees) == FALSE)) {
                all_raxml_superfine_trees[[tree]] = multi2di(all_raxml_superfine_trees[[tree]])
            }
        }

        # Reference tree block, each gene family has it's own reference tree that is the one generated by the simulation step. Along this tree all sequences are generated and then alligned and passed as input for main.nf pipeline.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree,as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and perform aother operation and then save the paralog tree to the specifc gene faimily directory
        ref_subtree        = unroot(ref_subtree)
        ref_subtree        = list(ref_subtree)
        class(ref_subtree) = "multiPhylo"
        subtree_out_file   = file.path(m, "results/rerun", paste0("ref_subtree_", basename(m), ".nwk"))
        # TODO remove this comment write.tree(ref_subtree, file=subtree_out_file)

        # update on the average branch length with the mean of the true/refernce paralog tree of the given gene family
        avg_branch_length = c(avg_branch_length, mean(ref_subtree[[1]]$edge.length))

        # Make a list containing all trees gathered so far and compute all possible pair comparisons among trees. There are 1001*1001-1001 unique pairewise comparisons ( a square matrix of size 1001 with 0 in diagonal ).
        # On this matrix (plotdata) all other measuramnts are done. This is a symmetric matrix (distance matrix)
        all_trees        = c(ref_subtree, all_units_trees, all_superfine_trees, all_units_raxml_trees, all_raxml_superfine_trees)
        class(all_trees) = "multiPhylo"
        all_vs_all_rf    = RF.dist(all_trees, normalize = TRUE)
        plotdata         = data.matrix(all_vs_all_rf)

        # all ME metrics to be computed superfine = SuperTree
        mean_rf           = c()
        mean_superfine_rf = c()

        # all ML metrics to be computed
        mean_raxml_rf           = c()
        mean_raxml_superfine_rf = c()

        # proceed throught the plotdata matrix taking from the second element one every ten up to 242. Basically proceed at chunkcs of 10 (samples). On those chuncks compute the mean and the Standard error. 
        # average all sample "values" whithin a unit. going from 250 value to 25 averages
        # This iterate only through the values of the comparison between the reference paralofg tree (ref_subtree)(first column/row of matrix) and the 250 paraogs trees of ME + SM (all_units_trees)
        for (i in seq(2, (length(orgs$V1)*10)+1, by=10)) {
            mean_rf = c(mean_rf, mean(plotdata[1, i:(i+9)]))
        }

        # This instead goes from row 252 to 492 of the plotdata matrix. It still goes in chuncks of 10 (samples). 
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ME + ST (all_superfine_trees)
        for (i in seq( (length(orgs$V1)*10)+2, (2*(length(orgs$V1)*10)+1), by=10 )) {
            mean_superfine_rf = c(mean_superfine_rf, mean(plotdata[1,i:(i+9)]))
        }

        # This for loop goes from 502 to 742. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + SM (all_units_raxml_trees)
        for (i in seq( (2*(length(orgs$V1)*10))+2, (3*(length(orgs$V1)*10)+1), by=10 )) {
            mean_raxml_rf = c(mean_raxml_rf, mean(plotdata[1,i:(i+9)]))
        }

        # This for loop goes from 752 to 992. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + ST (all_raxml_superfine_trees)
        for (i in seq( (3*(length(orgs$V1)*10))+2, (4*(length(orgs$V1)*10)+1), by=10)) {
            mean_raxml_superfine_rf = c(mean_raxml_superfine_rf, mean(plotdata[1,i:(i+9)]))
        }

        # collect all mean RF values 25 mean_rf, 25 mean_superfine_rf ecc.. for each family. One for each unit.
        collection_mean_rf      = c(mean_rf, mean_superfine_rf, mean_raxml_rf, mean_raxml_superfine_rf)

        # create the column names for the later dataframe, ME = minimum Evolutio, ML = Maximum Likelihood, SM = SuperMatrix , ST = SuperTree. names will be like ME_SM_unit_1
        units_names = c()
        for (i in 1:length(orgs$V1)) {
            units_names = c(units_names, paste("ME_SM", "unit", i, sep="_"), 
                            paste("ME_ST", "unit", i, sep="_"), 
                            paste("ML_SM", "unit", i, sep="_"), 
                            paste("ML_ST", "unit", i, sep="_"))
        }

        # natural_sort them or human readable sort, so that the order is  ME_SM, ME_ST, ML_SM and ML_ST
        mixedsort(units_names)

        # Finally update the "global" lists created at the beginning
        RF_avg_list      = list.append(RF_avg_list, collection_mean_rf)

    }
}



[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_fact

In [12]:
# transform the list of lists (RF_avg_list) into a dataframe (matrix) size 500 * 100 (25 units for all combinations of ME,ML and SM,ST)
RF_avg_as_df = ldply(RF_avg_list, recursive=FALSE)

# TODO see if standar error se was actually usefull
RF_df = round(RF_avg_as_df, digits=3)

# add the units name generated before as column names for the dataframe and save it to file
colnames(RF_df) = units_names
write.table(RF_df, file = "tables/avg_RF_between_ref_supermatrix_and_supertree.tsv", quote = FALSE, row.names = FALSE)

<a id="chapter-5-BigTree-VS-true-topologies"></a>

## Chapter 5 BigTree VS true topologies

TODO add description

In [14]:
# Trees from BigTree (Gene family tree) VS true topologies

mean_rf_subtrees_from_full_aln_vs_true_tree = c()
sd_rf_subtrees_from_full_aln_vs_true_tree = c()
RF_matrices = list()

collect_branch_len = c()
collect_diameter = c()
collect_branch_len_sub_from_ref = c()
collect_diameter_sub_from_ref = c()
collect_branch_len_big_trees = c()
collect_diameter_big_trees = c()

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get al the family subdirectories in each mqin.nf output run dir
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # this will contain the mean branch length for all true/reference paralog trees for each gene family
    avg_branch_length = c()

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # extract and work on all BigTree msa for all families. this MSA (500) is the input of the main.nf for simulated data workflow, whose outputs are analyzed in this notebook.
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 10 == 0) {
            print(m)
        }

        # each sequence in the MSA id mapping. Each sequence header is mapped to a new generic one, this file holds such mapping.
        full_aln_codepath = file.path(m, "results/rerun", "full_aln.code_name")
        full_aln_codefile = read.table(file = full_aln_codepath)

        # variable to store all species subtrees extracted from the Gene family MSA aka BigTree MSA
        all_species_raxml_subtrees_from_full_aln = c()

        # get the BigTree tree. full_aln=BigTree and unroot it 
        bigtree_file  = file.path(m, "results/rerun", paste0("RAxML_bestTree.", basename(m), "_full_aln_coded_raxml.nwk"))
        full_aln_tree = read.tree(bigtree_file)
        full_aln_tree = unroot(full_aln_tree)

        # update the branch len of the BigTrees vector. each tree has 375 tips (25 species * 15 paralog), so there are 747 (2*375-3)  branch length for each tree.
        collect_branch_len_big_trees = c(collect_branch_len_big_trees, full_aln_tree$edge.length)

        # update the diameter of the BigTrees. diameter=max distance between two tips in the tree. The distance is computed using the Patristic method.
        collect_diameter_big_trees   = c(collect_diameter_big_trees, max(distTips(full_aln_tree, method = 'patristic')))
        
        # for each tip in the tree (375) rename it's label according to the mapping of full_aln.code_name. essentially going from C181 -> Human_15
        for (j in 1:length(full_aln_tree$tip.label)) { 
            full_aln_tree$tip.label[j] = as.character(full_aln_codefile$V1[which(full_aln_codefile$V2 == full_aln_tree[["tip.label"]][j])])
        }

        # Now actually extract and work of subTree level. The subtrees are the paralog trees for a given species. That's why itarate over all species names and the change tps label naming.
        for (species in orgs$V1) {

            # For each species, the code searches the tip labels in the tree for those that start with the species name ("^" and perl = TRUE) and stores the corresponding indices in the tips_to_keep variable.
            tips_to_keep = grep(paste0("^", species), full_aln_tree$tip.label, perl = TRUE)

            # now that we have the inidces actually extrat the species paralog subtree. Unroot it.
            test_subtree = keep.tip(full_aln_tree, tips_to_keep)
            test_subtree = unroot(test_subtree)

            # rename the tips label again. going from Chicken_15 -> Seq_15. this for all species.
            test_subtree$tip.label = gsub(paste0(species, "_"), "Seq_", test_subtree$tip.label)

            # collect the subtrees branch length and the diameter as above.
            collect_branch_len = c(collect_branch_len, test_subtree$edge.length)
            collect_diameter = c(collect_diameter, max(distTips(test_subtree, method = 'patristic')))

            # change class of tree and add it to a vector
            test_subtree = as.multiPhylo(test_subtree)
            all_species_raxml_subtrees_from_full_aln = c(all_species_raxml_subtrees_from_full_aln, test_subtree)
        }
        class(all_species_raxml_subtrees_from_full_aln) = "multiPhylo"

        # read the reference tree (gene family tree) the one set as throuth and on which the sequences were generated along.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # refernce tree block similar to the for loop above
        for (species in orgs$V1) {
            
            # ectract tips like before and then extract subtree and unroot it. 
            # this is the refence paralog tree for each species and whithin one family all species have the same paralog tree as far as topology goes, what changes between this species paralog trees is their branch lengths.
            tips_to_keep = grep(paste0("^", species), ref_tree$tip.label, perl = TRUE)
            test_subtree = keep.tip(ref_tree, tips_to_keep)
            test_subtree = unroot(test_subtree)

            # get branch length and diameter
            collect_branch_len_sub_from_ref = c(collect_branch_len_sub_from_ref, test_subtree$edge.length)
            collect_diameter_sub_from_ref   = c(collect_diameter_sub_from_ref, max(distTips(test_subtree, method = 'patristic')))
        }

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree,as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and perform aother operation and then save the paralog tree to the specifc gene faimily directory
        ref_subtree        = unroot(ref_subtree)
        ref_subtree        = list(ref_subtree)
        class(ref_subtree) = "multiPhylo"

        # update on the average branch length with the mean of the true/refernce paralog tree of the given gene family
        avg_branch_length = c(avg_branch_length, mean(ref_subtree[[1]]$edge.length))

        # put all  BIgTree (big MSA + ML) species subtrees and the reference paralog tree into one single vector so Roubinson-Fould dictance can be computed.
        all_trees        = c(ref_subtree, all_species_raxml_subtrees_from_full_aln)
        class(all_trees) = "multiPhylo"

        # compute the RF dictance between 1 refernce paralog tree and all (25) species paralog tress extracted from BigTree.
        all_vs_all_rf = RF.dist(all_trees, normalize=TRUE)
        plotdata      = data.matrix(all_vs_all_rf)
        RF_matrices   = list.append(RF_matrices, plotdata)

        # do the average of all pair comparison between the species paralog trees and the reference tree. -c[1] means not consider the first column, because first row first column is RF between reference tree and itself
        mean_rf_subtrees_from_full_aln_vs_true_tree = c(mean_rf_subtrees_from_full_aln_vs_true_tree, mean(plotdata[1, -c(1)]))
        sd_rf_subtrees_from_full_aln_vs_true_tree   = c(sd_rf_subtrees_from_full_aln_vs_true_tree, sd(plotdata[1, -c(1)]))
    }
}

write.table(mean_rf_subtrees_from_full_aln_vs_true_tree, file = "tables/avg_RF_between_ref_big_tree_subtrees.tsv", quote=FALSE, row.names = FALSE, col.names = FALSE)

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_010.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_020.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/avg_branchlen_0.7_protein_family_030.sub"
[1] "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_fact

<a id='chapter-6-RF-distance-summaries'></a>

## Chapter 6 - RF distance summaries

todo add description

<a id='subchapter-6_1-Table-1'></a>

### Subchapter 6.1 - Table 1
todo add description.

In [16]:
# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/avg_RF_between_ref_supermatrix_and_supertree.tsv", header=T)

# do the mean and sd of the column. effectively getting a vector of size 100 filed with averages of averages and with ids like SM_ML_unit9
rf_df_new_means = colMeans(df_new)
sd_df_new_means = apply(df_new, 2, sd)

# Initialize empty list to hold the extracted method names and unit names values
methods = c("ME_SM", "ML_SM", "ME_ST", "ML_ST")
units   = 25

# Create an empty dataframe with row names for methods and columns for unit_1 to unit_25
avg_rf_df_method_unit           = data.frame(Method = methods, matrix(ncol = units, nrow = 4))
colnames(avg_rf_df_method_unit) = c("Method", paste0("unit_", 1:units))

# Fill the dataframe using the keys of the named vector of rf_df_new_means, these names are the same for the vector sd_df_new_means too.
for (key in names(rf_df_new_means)) {

  # Split the key into parts (method and unit)
  parts  = strsplit(key, "_unit_")[[1]]
  method = parts[1]
  unit   = as.numeric(parts[2])

  # Place the value in the correct row and column of the dataframe adding the standard deviation to it.
  avg_rf_df_method_unit[avg_rf_df_method_unit$Method == method, paste0("unit_", unit)] = paste0(round(rf_df_new_means[key], digit=2), "  sd  ", round(sd_df_new_means[key], digit=2))
}

# create the actual table with only unit1, unit25 difference values from above df
diff   = c(as.numeric(sub(" sd.*", "", avg_rf_df_method_unit$unit_1)) - as.numeric(sub(" sd.*", "", avg_rf_df_method_unit$unit_25)))
table1 = data.frame(Method = avg_rf_df_method_unit$Method,
                    "Unit 25"    = avg_rf_df_method_unit$unit_25,
                    "Unit 1"     = avg_rf_df_method_unit$unit_1,
                    "Difference" = diff )

# read the table compiled in chapter 5. So this cell does not depend on that one to run, it just needs the table file.
df_bigtree_new = read.table("tables/avg_RF_between_ref_big_tree_subtrees.tsv", header=F)

# make average of all family BigTree RF distances to their respective paralog tree. going from 500 values to 1.
rf_df_bigtree_new_mean = colMeans(df_bigtree_new)
rf_df_bigtree_new_sd   = apply(df_bigtree_new, 2, sd)

# add the above two values to table1
bigtree_row = c("BigTree", paste0(round(rf_df_bigtree_new_mean, digits = 2), "  sd  ", round(rf_df_bigtree_new_sd, digits = 2)), "Na", "Na")
table1      = rbind(table1, bigtree_row) 

avg_rf_df_method_unit

table1
#write.table(table1, file="Table1_average_RF.tsv",quote=FALSE, sep="\t",row.names=FALSE)

Method,unit_1,unit_2,unit_3,unit_4,unit_5,unit_6,unit_7,unit_8,unit_9,⋯,unit_16,unit_17,unit_18,unit_19,unit_20,unit_21,unit_22,unit_23,unit_24,unit_25
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ME_SM,0.47 sd 0.13,0.29 sd 0.13,0.26 sd 0.14,0.25 sd 0.14,0.25 sd 0.14,0.24 sd 0.14,0.24 sd 0.15,0.34 sd 0.13,0.28 sd 0.13,⋯,0.21 sd 0.13,0.2 sd 0.13,0.19 sd 0.13,0.19 sd 0.14,0.42 sd 0.13,0.27 sd 0.13,0.23 sd 0.12,0.22 sd 0.12,0.21 sd 0.12,0.2 sd 0.12
ML_SM,0.33 sd 0.13,0.27 sd 0.13,0.26 sd 0.14,0.25 sd 0.14,0.24 sd 0.14,0.24 sd 0.14,0.44 sd 0.13,0.3 sd 0.13,0.26 sd 0.13,⋯,0.2 sd 0.13,0.2 sd 0.13,0.19 sd 0.14,0.19 sd 0.14,0.31 sd 0.13,0.25 sd 0.12,0.23 sd 0.12,0.21 sd 0.12,0.21 sd 0.12,0.2 sd 0.13
ME_ST,0.37 sd 0.13,0.28 sd 0.14,0.26 sd 0.14,0.25 sd 0.14,0.24 sd 0.14,0.24 sd 0.14,0.47 sd 0.13,0.32 sd 0.13,0.27 sd 0.13,⋯,0.2 sd 0.13,0.2 sd 0.13,0.19 sd 0.14,0.19 sd 0.14,0.34 sd 0.13,0.26 sd 0.12,0.23 sd 0.12,0.21 sd 0.12,0.21 sd 0.12,0.2 sd 0.12
ML_ST,0.31 sd 0.14,0.27 sd 0.14,0.25 sd 0.14,0.25 sd 0.14,0.24 sd 0.14,0.24 sd 0.14,0.37 sd 0.13,0.29 sd 0.13,0.26 sd 0.13,⋯,0.2 sd 0.13,0.2 sd 0.14,0.19 sd 0.14,0.44 sd 0.13,0.29 sd 0.13,0.24 sd 0.12,0.22 sd 0.12,0.21 sd 0.12,0.2 sd 0.12,0.2 sd 0.12


Method,Unit.25,Unit.1,Difference
<chr>,<chr>,<chr>,<chr>
ME_SM,0.2 sd 0.12,0.47 sd 0.13,0.27
ML_SM,0.2 sd 0.13,0.33 sd 0.13,0.13
ME_ST,0.2 sd 0.12,0.37 sd 0.13,0.17
ML_ST,0.2 sd 0.12,0.31 sd 0.14,0.11
BigTree,0.18 sd 0.15,Na,Na


<a id="chapter-7-Running-times"></a>

## Chapter 7 - Running times

TODO add description 

In [6]:
# read the trace files for running times
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "trace_cpu_time.txt")
    tracefile      = read.table(tracefile_path, header = TRUE)
    
    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod  = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$native_id), c("name", "native_id", "realtime")]
    
    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)

    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$family_group = rep(basename(run), 100)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species"


In [7]:
# TODO make nicer and update with whatever is changed above

# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        638.12          67.76

[1] 588.3

[1] 63.85

<a id='subchapter-7_1-Figure-2'></a>

### Subchapter 7.1 - Figure 2

TODO add chapter link and description if necessary.

In [8]:
# TODO make this into the section of custom functions.
# Function to assign new values based on a mapping provided by the user
assign_mapped_values <- function(df, column_name, mapping, new_col_name) {
  # Check if mapping is a named vector or a list
  if (!is.vector(mapping) || is.null(names(mapping))) {
    stop("Mapping must be a named vector or list.")
  }
  
  # Get the unique values from the specified column
  unique_values = unique(df[[column_name]])
  
  # Check if the mapping contains all unique values
  if (!all(unique_values %in% names(mapping))) {
    stop("Mapping must include all unique values from the specified column.")
  }
  
  # Initialize a vector to store the assigned values
  assigned_values = vector("character", nrow(df))
  
  # Assign values based on the mapping
  for (i in seq_along(unique_values)) {
    assigned_values[df[[column_name]] == unique_values[i]] <- mapping[names(mapping) == unique_values[i]]
  }
  
  # Add the assigned values to the original dataframe
  df[[new_col_name]] = assigned_values
  
  return(df)
}

In [9]:
#
# Cpu time vs Method (SM + ML and Bigtree) divided per group (by sequence length)
#

# assign to each family_group a mapping values  (as new column). basically insert the length of the sequences in each family as an additional column in the dataframe.
# Remember each family in the same group have the same length of sequence generated, the MSA though might differ in length due to gaps.
mapping = c("rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1\nlen = 32",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2\nlen = 46",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3\nlen = 50",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4\nlen = 60",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5\nlen = 82")
fin_tracefile_mod_reassigned = assign_mapped_values(fin_tracefile_mod_wide, "family_group", mapping, "length")

# reorder the columns to have the numerical values at the end
fin_tracefile_mod_reassigned = fin_tracefile_mod_reassigned[, c("name", "family_group", "length", "Big_Tree", "Supermatrix_ML")]

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_melted           = melt(fin_tracefile_mod_reassigned)


# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_melted$value     = fin_tracefile_mod_melted$value / 1000
colnames(fin_tracefile_mod_melted) = c("name", "family_group", "length", "Method", "Time")

# create a table with 5 pairs of boxplot. Group1, 2.. ecc.. each group has 100 families and two running time (realtime) mesuraments for the BigTree and SM + ML approaches.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_melted, aes(x=length, y=Time, fill=Method)) +
    geom_boxplot(show.legend = TRUE) +
    theme_light() +
    labs(x = "sequence length",
       y = "CPU time (sec)",
       fill = "Method")
ggsave(filename="figures/CPU_time_per_group_BigTree_SM_ML.png", plot=p, dpi = "retina")

Using name, family_group, length as id variables

Saving 7 x 7 in image


### Suplementary Fig todo put number of supplementary figure

In [14]:
#
# Cpu time vs Method ratio (Bigtree / SM + ML ) boxplot of each family ratio by group (x axis groups y ratio Big/SM) 
# 

# start by taking the already present info for the families and their grouping from fig 2 cell and compute ratio Bigtree / SM + ML
# Create a new column with the division of Big_Tree by Supermatrix_ML running times for each family (500)
# and then remove the original columns Big_Tree Supermatrix_ML
fin_tracefile_mod_ratio = fin_tracefile_mod_reassigned %>%
  mutate(fold_ratio = Big_Tree / Supermatrix_ML) %>%
  select(-Big_Tree, -Supermatrix_ML)

# create a table with 5 boxplots. Group1, 2.. ecc.. each group has 100 families and  the ratio of two running time (realtime) mesuraments for the BigTree and SM + ML approaches.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_ratio, aes(x=length, y=fold_ratio)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    labs(x = "sequence length",
       y = "CPU time fold ratio (BigTree / Supermatrix_ML)")
ggsave(filename="figures/CPU_time_per_group_BigTree_SM_ML_ratio.png", plot=p, dpi = "retina")

Saving 7 x 7 in image


In [169]:
#
# Cpu time vs Method (SM + ML and Bigtree)   OLD VERSION
#

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name", "family_group", "Method", "Time")

# crate a table with two boxplots (x axis) with Method columns as values and on the y the running times.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel. 
p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method, y=Time, fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="figures/CPU_time_per_method.png", plot=p, dpi = "retina")

Using name, family_group as id variables

Saving 7 x 7 in image


<br><br><br>
# USEFULL STUFF BUT NOT INCLUDED IN PAPER
<br><br><br>

### OLD version all families toghther

### TODO make the following better or remove it

In [48]:
# update the trace matrix with the length of the input msa
library("Biostrings") 
fin_tracefile_mod_with_len = fin_tracefile_mod_wide

# for each family go and read the input MSA (input of main.nf) to get length of MSA (number of columns).
# Iterate over each row of the dataframe
for (i in 1:nrow(fin_tracefile_mod_with_len)) {
    # Construct the full file path using group (directory) and name (filename) to read the input msa
    # TODO make it not depend on the rerun_bla_bla direcrtory
    input_msa_path = file.path(paste0(parent_of_all_dir, "rerun_25_species_corrected_branches"), fin_tracefile_mod_with_len$family_group[i], paste0(gsub("[()]", "", fin_tracefile_mod_with_len$name[i]), ".ma"))

    # read the msa in fasta format and get the length of the first sequence (aka length of the alignment)
    msa_file = readAAStringSet(input_msa_path)
    msa_len  = length(msa_file[[1]])
    
    # add the length to the table 
    fin_tracefile_mod_with_len$length[i] = msa_len
}

In [4]:
#
# Cpu time vs Method (SM + ML and Bigtree) ratio plot of each family (x axis number columns BigMSA) 
# 

# Create a new column with the division of Big_Tree by Supermatrix_ML running times for each family (500)
# and then remove the original columns Big_Tree Supermatrix_ML
fin_tracefile_mod_ratio = fin_tracefile_mod_with_len %>%
  mutate(fold_ratio = Big_Tree / Supermatrix_ML) %>%
  select(-Big_Tree, -Supermatrix_ML)

# create a table with on x axis the length of the MSA (n columns)  and on y the fold ratio (BigTree / SM + ML) of running times. The plot will have now 500 points.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_ratio, aes(x=length, y=fold_ratio)) +
    geom_point(color = 'blue', size = 2, alpha = 0.6) +  # Add points
    theme_light() +
    labs(x = "MSA length (n# columns)",
       y = "fold ratio (BigTree / Supermatrix_ML)")
ggsave(filename="figures/CPU_time_scatterplot_msa_len_BigTree_SM_ML.png", plot=p, dpi = "retina")

ERROR: Error: object 'fin_tracefile_mod_with_len' not found


### TODO the below was just a 10 familoy trial make it go away 

In [109]:
# TODO make this into the section of custom functions.
# Function to convert to milliseconds
convert_to_ms <- function(time_str) {
    # Extract components using regular expressions
    millis = "0"
    seconds = "0"
    minutes = "0"
    if (str_detect(time_str, "\\d+(\\.\\d+)?(?=ms)")) {
        millis <- str_extract(time_str, "\\d+(\\.\\d+)?(?=ms)")
    } else {
        seconds <- str_extract(time_str, "\\d+(\\.\\d+)?(?=s)")
        minutes <- str_extract(time_str, "\\d+(\\.\\d+)?(?=m)")
    }
    
    # Convert each component to milliseconds (coercing to numeric where needed)
    seconds_ms <- as.numeric(seconds) * 1000
    minutes_ms <- as.numeric(minutes) * 60 * 1000
    millis_ms <- as.numeric(millis)
    
    # Sum all components and handle NAs by treating them as 0
    total_ms <- sum(c(seconds_ms, minutes_ms, millis_ms), na.rm = TRUE)
    return(total_ms)
}

In [120]:
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "pipeline_info/execution_trace_2024-09-27_17-39-02.txt")
    tracefile      = read.table(tracefile_path, sep = "\t", header = TRUE)

    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$name), c("name", "realtime")]

    # Use strsplit to split the 'name' column by whitespace
    split_name    = strsplit(as.character(tracefile_mod$name), " ")

    # Create new columns
    tracefile_mod$name       = sapply(split_name, `[`, 2) # First part 
    tracefile_mod$native_id  = sapply(split_name, `[`, 1) # Second part 

    # Reorder columns so 'native_id' becomes the second column
    tracefile_mod = tracefile_mod[, c("name", "native_id", "realtime")]

    # trasform from human readable time to millisecond time
    tracefile_mod$realtime = sapply(tracefile_mod$realtime, convert_to_ms)

    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)
    
    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$group = rep(basename(run), 10)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "results"


In [121]:
# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        195.40          23.47

[1] 196

[1] 21.87

In [122]:

fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name","group","Method","Time")

p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method,y=Time,fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="CPU_time_per_method_10_fam.png", plot=p, dpi = "retina")

Using name, group as id variables

Saving 7 x 7 in image
